In [ ]:
import math
import numpy as np

from src.options import GeometricBrownianMotion

from src.options import Option

In [ ]:
def feature_maps(simulated_paths: list[list[float]],
                 K: float,
                 T: float,
                 num_paths: int,
                 num_steps: int
                 ) -> np.ndarray:
    # Нормализация симулированных путей относительно цены исполнения опциона
    S = np.array(simulated_paths).flatten() / K

    # Создание матрицы признаков для аппроксимации value function
    fs = np.vstack([
        np.ones_like(S),  # константный
        np.exp(-S / 2),
        np.exp(-S / 2) * (1 - S),
        np.exp(-S / 2) * (1 - 2 * S + 0.5 * S * S)
    ])

    # Создание временных признаков
    t = np.arange(num_steps)
    dt = T / num_steps
    ft = np.vstack([
        np.sin(0.5 * math.pi * (1 - t / num_steps)),
        np.log(T - t * dt),
        (t / num_steps) ** 2
    ])
    # Добавление столбца для всех путей
    ft = np.hstack([ft, np.array([0, 0, 1]).reshape(3, 1)])
    ft = np.tile(ft, num_paths)

    # Комбинация признаков цен и времени
    return np.vstack([fs, ft])

In [4]:
def price_option(option: Option,
                 risk_free_rate: float,
                 volatility: float,
                 time_to_maturity_in_years: float,
                 num_paths: int = 5000,
                 num_steps: int = 50,
                 # for policy iteration
                 tol: float = 1e-6,
                 maxiter: int = 20,
                 seed: int | None = None) -> float:
    dt = time_to_maturity_in_years / num_steps
    drift = risk_free_rate - option.dividend_yield

    simulated_paths = GeometricBrownianMotion.simulate(
        init=option.underlying.spot_price,
        mu=drift,
        sigma=volatility,
        dt=dt,
        num_steps=num_steps,
        num_paths=num_paths,
        seed=seed
    )

    total_num_price_points = num_paths * (num_steps + 1)

    phi = feature_maps(
        simulated_paths=simulated_paths,
        K=option.strike_price,
        T=time_to_maturity_in_years,
        num_paths=num_paths,
        num_steps=num_steps
    )
    assert phi.shape[1] == total_num_price_points

    # Вычисление выплат по опционам
    payoffs = np.array([
        option.payoff(S, time_to_maturity_in_years - t * dt)
        for path in simulated_paths for (t, S) in enumerate(path)
    ])
    assert payoffs.shape == (total_num_price_points,)

    # Основная матрица для LSPI
    k = phi.shape[0]
    gamma = math.exp(-risk_free_rate * dt)  # дисконт фактор

    # матрица грамма?
    A = phi @ phi.T
    assert A.shape == (k, k)
    # почему такая размерность у А и phi?

    # исключаем конечные состояния каждой траектории
    phi_dash = phi.reshape((k, num_paths, num_steps + 1))[:, :, :-1].reshape((k, -1))

    def LSTDQ(w: np.ndarray) -> np.ndarray:
        # Рассчитываем Q-value, используя текущий вектор весов w и матрицу признаков phi.
        # Q-value ограничена снизу нулем с помощью максимума
        Q = np.maximum(w.dot(phi), 0)
        assert Q.shape == (total_num_price_points,)

        # Определяем условие продолжения для каждого временного шага каждого пути:
        # Если payoff < Q-value, предполагается продолжение.
        to_continue = (payoffs < Q)

        # Фильтруем матрицу признаков, чтобы получить признаки, где продлжаем
        psi = (to_continue * phi).reshape((k, num_paths, num_steps + 1))[:, :, 1:].reshape((k, -1))

        # Вычисляем промежуточную матрицу a, используемую в формуле обновления LSTDQ
        # Эта матрица представляет собой произведение  признаков по всем путям и шагам, где продолжаем
        # подумать про а маленькое
        a = phi_dash[:, None, :] * psi[None, :, :]
        assert a.shape == (k, k, num_paths * num_steps), a.shape

        # Вектор g представляет вознаграждения за немедленную остановку в данный момент (т.е. где payoff >= Q-value)
        g = ((~to_continue) * payoffs).reshape((num_paths, num_steps + 1))[:, 1:].reshape(-1)
        # b отражает суммарные ожидаемые вознаграждения, скорректированные с учётом коэффициента дисконтирования, для тех шагов, 
        # где было принято решение остановиться (?)
        b = gamma * (phi_dash @ g)
        assert b.shape == (k,)

        # Решаем задачу наименьших квадратов для нахождения нового вектора весов:
        # Корректируем матрицу A, вычитая взвешенную сумму матриц из a, затем решаем Ax = b.
        return np.linalg.lstsq(A - gamma * np.sum(a, axis=2), b, rcond=None)[0]

    def LSPI() -> np.ndarray:
        # Итеративное улучшение политики через LSTDQ
        # Инициализируем вектор весов нулями.
        # Повторяем цикл улучшения политики до maxiter раз.
        w = np.zeros(k)
        for i in range(maxiter):
            # Обновляем вектор весов с помощью метода LSTDQ, который улучшает политику.
            new_w = LSTDQ(w)
            if np.linalg.norm(new_w - w) < tol:
                return new_w
            w = new_w
        return w

    def greedy_exercise_payoff(w: np.ndarray):
        Q = np.maximum(w.dot(phi), 0)
        to_exercise = (payoffs >= Q)

        def first_nonzero(a: np.ndarray) -> float:
            nz = np.nonzero(a)
            return a[nz][0] if len(a[nz]) > 0 else 0

        payoff_from_best_policy = np.apply_along_axis(
            first_nonzero,
            axis=1,
            arr=(to_exercise * payoffs).reshape((num_paths, num_steps + 1))
        )
        assert payoff_from_best_policy.shape == (num_paths,)

        return np.mean(payoff_from_best_policy)

    # поиск оптимальной остановки
    w = LSPI()

    # рассчет ожидаемого payoff`a
    expected_payoff = greedy_exercise_payoff(w)
    immediate_payoff = option.payoff(option.underlying.spot_price, time_to_maturity_in_years)
    return round(max(expected_payoff, immediate_payoff), 2)